<a href="https://colab.research.google.com/github/jon-nowacki/Optimization-Models/blob/main/ConcreteModels_ErrorHandling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Step 1 - Install Pyomo</h1>
Start by installing the pyomo package.  You will need to do this at the beginning of every session.

https://www.youtube.com/watch?v=AOEG_MEwSDA&t=343s

In [ ]:
!pip install -q pyomo

<h1>Step 2 - Install GLPK</h1>
Next, install GLPK. You will need to do this at the beginning of every session as well.

In [ ]:
!apt-get install -y -qq glpk-utils

<h1> Step 3a - Here's the CSL Concrete Model Formulation</h1>

The original problem description of the CSL model can be found in Winston, W. (2004) Operations Research: Applications and Algorithms, Fourth Edition, Brooks/Cole Cengage Learning.

The concrete version of this model would look like this:

<h3> Decision Variables </h3>
T1 = number of trainees to train in month 1 <br>
T2 = number of trainees to train in month 2<br>
T3 = number of trainees to train in month 3<br>
T4 = number of trainees to train in month 4<br>
T5 = number of trainees to train in month 5<br>

E1 = number of experienced technicians at the beginning of month 1 <br>
E2 = number of experienced technicians at the beginning of month 2<br>
E3 = number of experienced technicians at the beginning of month 3<br>
E4 = number of experienced technicians at the beginning of month 4<br>
E5 = number of experienced technicians at the beginning of month 5<br>

minimize LabourCost = 1000(T1+T2+T3+T4+T5)+2000(E1+E2+E3+E4+E5)

<h3> Subject To </h3>
160*E1 - 50*T1 >=6000  (meet required hours in Month 1) <br>
160*E2 - 50*T2 >=7000  (meet required hours in Month 2) <br>
160*E3 - 50*T3 >=8000  (meet required hours in Month 3) <br>
160*E4 - 50*T4 >=9500  (meet required hours in Month 4) <br>
160*E5 - 50*T5 >=11000 (meet required hours in Month 4) <br>

E1=50 (starting number of experienced technicians)<br>
E2=0.95*E1 +T1 (experienced technicians at beginning of month 2)<br>
E3=0.95*E2 + T2  (experienced technicians at beginning of month 3)<br>
E4=0.95*E3 + T3 (experienced technicians at beginning of month 4)<br>
E5=0.95*E4 + T4  (experienced technicians at beginning of month 5)<br>

T1, T2, T3, T4, T5 >=0 <br>
E1, E2, E3, E4, E5 >=0 <br>



<h1> Step 3b - Here's the CSL Concrete Model in Pyomo</h1>

<img src="https://drive.google.com/uc?export=view&id=1PFzrsQeMPsw6fzlSn0gMCKszGVM99bX3" width="800"/>


**NOTE:** 0.95 is to account for attrition rate of 5%

<img src="https://drive.google.com/uc?export=view&id=1wmrvl3V8sKxXm4NXWGpWWIki6TunOjx0" width="800"/>






**Key Notes**
* X=Y: sets X to Y
* X==Y: says X must be equal to Y

In [ ]:
from pyomo.environ import *

#create model object as a concrete model
model= ConcreteModel ()

#declare variables
model.T1 = Var(within=NonNegativeReals) #no. of trainees to train in month 1
model.T2 = Var(within=NonNegativeReals) #no. of trainees to train in month 2
model.T3 = Var(within=NonNegativeReals) #no. of trainees to train in month 3
model.T4 = Var(within=NonNegativeReals) #no. of trainees to train in month 4
model.T5 = Var(within=NonNegativeReals) #no. of trainees to train in month 5

model.E1 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 1
model.E2 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 2
model.E3 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 3
model.E4 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 4
model.E5 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 5

#objective function - Minimize the total labor costs
model.MinimizeLabourCost = Objective (expr = 1000*model.T1+1000*model.T2+1000*model.T3+1000*model.T4+
1000*model.T5+2000*model.E1+2000*model.E2+2000*model.E3+2000*model.E4+2000*model.E5,sense= minimize)

# Constraints to make sure we meed demand each month
model.month1Demand = Constraint (expr = 160*model.E1 - 50*model.T1 >=6000)
model.month2Demand = Constraint (expr = 160*model.E2 - 50*model.T2 >=7000)
model.month3Demand = Constraint (expr = 160*model.E3 - 50*model.T3 >=8000)
model.month4Demand = Constraint (expr = 160*model.E4 - 50*model.T4 >=9500)
model.month5Demand = Constraint (expr = 160*model.E5 - 50*model.T5 >=11000)

#Constraints to keep track of the number of experienced technicians at the beginning of each month
model.experMonth1 = Constraint (expr = model.E1==50)
model.experMonth2 = Constraint (expr = model.E2==0.95*model.E1 + model.T1)
model.experMonth3= Constraint (expr = model.E3==0.95*model.E2 + model.T2)
model.experMonth4= Constraint (expr = model.E4==0.95*model.E3 + model.T3)
model.experMonth5= Constraint (expr = model.E5==0.95*model.E4 + model.T4)



<h1> Step 4 - Solve the Model </h1>
Next we must create a SolverFactory that uses the glpk solver. Then solve the model and display the results.

In [ ]:
#Lines to run the model from within a python IDE
optimizer = SolverFactory("glpk",executable='/usr/bin/glpsol')     #creates an optimizer object that uses the glpk package installed to your usr/bin.
optimizer.solve(model)                #tells your optimizer to solve the model object
model.display()                       #writes results to the console so you can see them

Model unknown

  Variables:
    T1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    T2 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 8.45316809406301 :  None : False : False : NonNegativeReals
    T3 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 11.4501379010016 :  None : False : False : NonNegativeReals
    T4 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 9.51807228915664 :  None : False : False : NonNegativeReals
    T5 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    E1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
      

# Unbounded


In [ ]:
from pyomo.environ import *

#create unbounded object as a concrete unbounded
unbounded= ConcreteModel ()

#declare variables
unbounded.T1 = Var() #no. of trainees to train in month 1
unbounded.T2 = Var() #no. of trainees to train in month 2
unbounded.T3 = Var() #no. of trainees to train in month 3
unbounded.T4 = Var() #no. of trainees to train in month 4
unbounded.T5 = Var() #no. of trainees to train in month 5

unbounded.E1 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 1
unbounded.E2 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 2
unbounded.E3 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 3
unbounded.E4 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 4
unbounded.E5 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 5

#objective function - Minimize the total labor costs
unbounded.MinimizeLabourCost = Objective (expr = 1000*unbounded.T1+1000*unbounded.T2+1000*unbounded.T3+1000*unbounded.T4+
1000*unbounded.T5+2000*unbounded.E1+2000*unbounded.E2+2000*unbounded.E3+2000*unbounded.E4+2000*unbounded.E5,sense= minimize)

# Constraints to make sure we meed demand each month
unbounded.month1Demand = Constraint (expr = 160*unbounded.E1 - 50*unbounded.T1 >=6000)
unbounded.month2Demand = Constraint (expr = 160*unbounded.E2 - 50*unbounded.T2 >=7000)
unbounded.month3Demand = Constraint (expr = 160*unbounded.E3 - 50*unbounded.T3 >=8000)
unbounded.month4Demand = Constraint (expr = 160*unbounded.E4 - 50*unbounded.T4 >=9500)
unbounded.month5Demand = Constraint (expr = 160*unbounded.E5 - 50*unbounded.T5 >=11000)

#Constraints to keep track of the number of experienced technicians at the beginning of each month
unbounded.experMonth1 = Constraint (expr = unbounded.E1==50)
unbounded.experMonth2 = Constraint (expr = unbounded.E2==0.95*unbounded.E1 + unbounded.T1)
unbounded.experMonth3= Constraint (expr = unbounded.E3==0.95*unbounded.E2 + unbounded.T2)
unbounded.experMonth4= Constraint (expr = unbounded.E4==0.95*unbounded.E3 + unbounded.T3)
unbounded.experMonth5= Constraint (expr = unbounded.E5==0.95*unbounded.E4 + unbounded.T4)



NOTE: Objective gets "none" instead of error.

In [ ]:
#Lines to run the model from within a python IDE
optimizer = SolverFactory("glpk",executable='/usr/bin/glpsol')     #creates an optimizer object that uses the glpk package installed to your usr/bin.
optimizer.solve(unbounded)                #tells your optimizer to solve the model object
unbounded.display()                       #writes results to the console so you can see them

ERROR:pyomo.common.numeric_types:evaluating object as numeric value: T1
    (object: <class 'pyomo.core.base.var.ScalarVar'>)
No value for uninitialized NumericValue object T1
ERROR:pyomo.common.numeric_types:evaluating object as numeric value: MinimizeLabourCost
    (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
No value for uninitialized NumericValue object T1


Model unknown

  Variables:
    T1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    T2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    T3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    T4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    T5 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    E1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    E2 : Size=1, Index=None
        Key  : Lower :


# Install Catches

In [ ]:
from pyomo.opt import SolverStatus, TerminationCondition


**Delete the "within=NonNegativeReals" from T1 - T5 to get it to work**

In [ ]:
#create unbounded object as a concrete unbounded
unbounded= ConcreteModel ()

#declare variables
unbounded.T1 = Var(within=NonNegativeReals) #no. of trainees to train in month 1
unbounded.T2 = Var(within=NonNegativeReals) #no. of trainees to train in month 2
unbounded.T3 = Var(within=NonNegativeReals) #no. of trainees to train in month 3
unbounded.T4 = Var(within=NonNegativeReals) #no. of trainees to train in month 4
unbounded.T5 = Var(within=NonNegativeReals) #no. of trainees to train in month 5

unbounded.E1 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 1
unbounded.E2 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 2
unbounded.E3 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 3
unbounded.E4 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 4
unbounded.E5 = Var(within=NonNegativeReals) #no. of experience techs at the beginning of month 5

#objective function - Minimize the total labor costs
unbounded.MinimizeLabourCost = Objective (expr = 1000*unbounded.T1+1000*unbounded.T2+1000*unbounded.T3+1000*unbounded.T4+
1000*unbounded.T5+2000*unbounded.E1+2000*unbounded.E2+2000*unbounded.E3+2000*unbounded.E4+2000*unbounded.E5,sense= minimize)

# Constraints to make sure we meed demand each month
unbounded.month1Demand = Constraint (expr = 160*unbounded.E1 - 50*unbounded.T1 >=6000)
unbounded.month2Demand = Constraint (expr = 160*unbounded.E2 - 50*unbounded.T2 >=7000)
unbounded.month3Demand = Constraint (expr = 160*unbounded.E3 - 50*unbounded.T3 >=8000)
unbounded.month4Demand = Constraint (expr = 160*unbounded.E4 - 50*unbounded.T4 >=9500)
unbounded.month5Demand = Constraint (expr = 160*unbounded.E5 - 50*unbounded.T5 >=11000)

#Constraints to keep track of the number of experienced technicians at the beginning of each month
unbounded.experMonth1 = Constraint (expr = unbounded.E1==50)
unbounded.experMonth2 = Constraint (expr = unbounded.E2==0.95*unbounded.E1 + unbounded.T1)
unbounded.experMonth3= Constraint (expr = unbounded.E3==0.95*unbounded.E2 + unbounded.T2)
unbounded.experMonth4= Constraint (expr = unbounded.E4==0.95*unbounded.E3 + unbounded.T3)
unbounded.experMonth5= Constraint (expr = unbounded.E5==0.95*unbounded.E4 + unbounded.T4)

#Lines to run the model from within a python IDE
optimizer = SolverFactory("glpk",executable='/usr/bin/glpsol')     #creates an optimizer object that uses the glpk package installed to your usr/bin.


In [ ]:
results=optimizer.solve(unbounded)                #tells your optimizer to solve the model object


if(results.solver.status==SolverStatus.ok) and (results.solver.termination_condition.optimal):
  #unbounded.display()                       #writes results to the console so you can see them
  print("\n")
  print("Number of trainees in month 1:",unbounded.T1())
  print("Total cost will be:", unbounded.MinimizeLabourCost())
  totalTrainees = unbounded.T1()+unbounded.T2()+unbounded.T3()+unbounded.T4()+unbounded.T5()
  print("Total cost will be:", totalTrainees)
  print("\t How many Trainees to hire")
  print("\t Month 1", unbounded.E1())
  print("\t Month 2", unbounded.E2())
  print("\t Month 3", unbounded.E3())
  print("\t Month 4", unbounded.E4())

elif(results.solver.termination_condition==TerminationCondition.infeasible) or results.solver.termination_condition==TerminationCondition.other:
  print("MODEL INFEASIBLE. Check Constraints")

else:
  print("Solver status", results.solver.status)
  print("Termination Condition", results.solver.termination_condition)



Number of trainees in month 1: 0.0
Total cost will be: 593776.50965307
Total cost will be: 29.421378284221248
	 How many Trainees to hire
	 Month 1 50.0
	 Month 2 47.5
	 Month 3 53.578168094063
	 Month 4 62.3493975903614


# Not in Video

1.   How many trainees i'm going to be training
2.   how many experienced technicians trained each month
3.   Total cost will be